# Import Libraries

In [11]:
import numpy as np
import pandas as pd
import spacy

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

print ("PyTorch version:[%s]."%(torch.__version__))
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print ("device:[%s]."%(device))

PyTorch version:[1.13.1+cu117].
device:[cuda:0].


# Make test data

In [38]:
nlp = spacy.load("en_core_web_md")

test_data = [
    "This is a positive sentence with good vibes.",
    "I'm not sure about this, it could be either positive or negative.",
    "Negative sentiment is evident in this statement.",
    "The weather is great today!",
]

labels = [1, 0, -1, 1]

def get_sentence_embedding(sentence):
    doc = nlp(sentence)
    return np.mean([token.vector for token in doc if token.has_vector], axis=0)

In [42]:
X_train = np.array([get_sentence_embedding(sentence) for sentence in test_data])

## check data' shape

In [51]:
print(f"X_train's shape: {X_train.shape}")

X_train's shape: (4, 300)


# Transformer

## MHA

In [2]:
class MultiHeadedAttention(nn.Module):
    def __init__(self,d_feat=300,n_head=5,actv=F.relu,device=None):

        super(MultiHeadedAttention, self).__init__()
        self.d_feat = d_feat
        self.n_head = n_head    #equal with d_k
        self.d_head = d_feat // n_head
        if self.d_head % self.n_head !=0:
            raise ValueError
        self.actv = actv

        self.lin_k = nn.Linear(self.d_feat, self.d_feat)
        self.lin_q = nn.Linear(self.d_feat, self.d_feat)
        self.lin_v = nn.Linear(self.d_feat, self.d_feat)

    def forward(self, Q, K, V):

        Query = self.lin_q(Q)
        Key = self.lin_k(K)
        Value = self.lin_v(V)

        #입력 쿼리는 n * s * d: batch * sequence_length * embedding length
        batch_size = Query.shape[0]
        Queries = Query.view(batch_size, -1, self.d_head, self.d_feat)
        Keys = Key.view(batch_size, -1, self.d_head, self.d_feat)
        Values = Value.view(batch_size, -1, self.d_head, self.d_feat)

        attention_score = torch.mat(Queries, torch.permute(Keys, 0, 1, 3, 2))
        attention_score = (attention_score / self.n_head) * Value
        attention_score = F.softmax(attention_score, dim = 0)

        

NameError: name 'nn' is not defined

In [76]:
torch.matmul(torch.tensor([[[1.,2.,3.,4.,5.,6.]]]),torch.ones(4, 6, 5))

tensor([[[21., 21., 21., 21., 21.]],

        [[21., 21., 21., 21., 21.]],

        [[21., 21., 21., 21., 21.]],

        [[21., 21., 21., 21., 21.]]])

In [74]:
torch.ones(4, 6, 5).shape

torch.Size([4, 6, 5])